# Linear programming approach

## Imports

In [1]:
import numpy as np
from pulp import *

from utils import Instance, PuLP_Problem, Gurobi_Problem

%load_ext autoreload
%autoreload 2

## Problem statement

**Constraints**

Pre-emption not allowed, full running time for each task:
$$\forall i \in \mathcal{I}, C_i = B_i + p_i$$

Release time:
$$\forall i \in \mathcal{I}, B_i \ge r_{j(i)}$$

Tasks have to be completed in order for each job:
$$\forall j \in \mathcal{J}, \forall i < i' \in S_j, B_{i'} \ge C_i$$

Two operators or two machines cannot handle more than one task at once:
$$\forall i \ne i' \in \mathcal{I}\,\text{s.t.}\,m_{i'}=m_{i}\,\text{or}\,o_{i'}=o_i, B_{i'} \notin \{B_i, \dots, B_i+p_i-1\}$$

**Objective**

$$\forall j \in \mathcal{J}, T_j := \max(0, C_j - d_j)$$
$$\forall j \in \mathcal{J}, U_j := \mathbb{1}_{C_j > d_j}$$

$$\min \sum_{j \in \mathcal{J}} w_j (C_j + \alpha U_j + \beta T_j)$$

## Greedy sub-optimal solution

In [2]:
instance_name = "tiny"
inst = Instance(instance_name)
inst.load(f"instances/{instance_name}.json")
print(f"J={inst.J} (number of jobs)\n" +
      f"I={inst.I} (number of tasks)\n" +
      f"M={inst.M} (number of machines)\n" +
      f"O={inst.O} (number of operators)\n" +
      f"α={inst.alpha} (unit penalty)\n" +
      f"β={inst.beta} (tardiness)")

J=5 (number of jobs)
I=25 (number of tasks)
M=8 (number of machines)
O=8 (number of operators)
α=6 (unit penalty)
β=1 (tardiness)


In [3]:
with open(f"solutions/KIRO-{instance_name}.json", 'rb') as f:
    sol = json.load(f)
for d in sol:
    task = inst.tasks[d['task']]
    task.B = d['start']
    task.C = d['start'] + task.p
print(f"saved file solution cost: {inst.cost()}")
for task in inst.tasks.values():
    task.B = None
    task.C = None

saved file solution cost: 1067


In [4]:
inst.greedy_solve()
print(f"greedy heuristic cost: {inst.cost()}")

greedy heuristic cost: 537


## Modeling - with PuLP

In [5]:
pulp_prob = PuLP_Problem(inst)

In [6]:
pulp_prob.generate_problem()

Generating PuLP problem for tiny...
Adding jobs/tasks variables and constraints...
Adding machines and operators variables and constraints...
Adding objective function...
PuLP problem generated.


In [7]:
pulp_prob.show_info()

Problem tiny has 1675 variables and 2311 constraints


In [8]:
pulp_prob.savefile()

Problem saved to lp_problems/pulp_tiny.mps


In [9]:
listSolvers(onlyAvailable=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-13
No parameters matching '_test' found


['GLPK_CMD', 'GUROBI', 'GUROBI_CMD', 'PULP_CBC_CMD', 'COIN_CMD']

In [10]:
pulp_prob.inst.greedy_solve()
pulp_prob.warmup()

In [11]:
pulp_prob.set_solver(PULP_CBC_CMD(msg=True, warmStart=True))

In [12]:
pulp_prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/louis/anaconda3/envs/perso/lib/python3.11/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/323da077329447eeaf4604c6c09c45a2-pulp.mps mips /tmp/323da077329447eeaf4604c6c09c45a2-pulp.mst timeMode elapsed branch printingOptions all solution /tmp/323da077329447eeaf4604c6c09c45a2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 2316 COLUMNS
At line 13273 RHS
At line 15585 BOUNDS
At line 17261 ENDATA
Problem MODEL has 2311 rows, 1675 columns and 7591 elements
Coin0008I MODEL read with 0 errors
opening mipstart file /tmp/323da077329447eeaf4604c6c09c45a2-pulp.mst.
MIPStart values read for 1675 variables.
Option for timeMode changed from cpu to elapsed
Continuous objective value is 441 - 0.03 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 241 strengthened rows, 0 substitutions
Cgl0004I processed model has 2280 rows, 1654 columns (1654 integer (1599 of which binary))

In [13]:
pulp_prob.show_status()

Problem status: Optimal
Obective value: 465.0


## Modeling - with Gurobi

In [14]:
instance_name = "medium"
inst = Instance(instance_name)
inst.load(f"instances/{instance_name}.json")

In [15]:
gurobi_prob = Gurobi_Problem(inst)

In [16]:
gurobi_prob.generate_problem()

Generating Gurobi problem for medium...
Adding jobs/tasks variables and constraints...
Adding machines and operators variables and constraints...
Adding objective function...
Gurobi problem generated.


In [12]:
gurobi_prob.savefile()

Problem saved to lp_problems/gurobi_large.mps


In [5]:
gurobi_prob.inst.greedy_solve()
gurobi_prob.warmup()

PreSolve (more aggressive application of presolve takes more time, but can sometimes lead to a significantly tighter model):
- -1: automatic setting
- 0: off
- 1: conservative
- 2: aggressive

Method (algorithm used to solve the initial root relaxation of the MIP model):
- -1: automatic
- 0: primal simplex
- 1: dual simplex
- 2: barrier
- 3: concurrent
- 4: deterministic concurrent
- 5: deterministic concurrent simplex

In [6]:
gurobi_prob.m.Params.PreSolve = 2
gurobi_prob.m.Params.Method = -1
gurobi_prob.m.Params.TimeLimit = 30*60

Set parameter Presolve to value 2
Set parameter TimeLimit to value 1800


In [7]:
gurobi_prob.solve()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 248965 rows, 167724 columns and 830244 nonzeros
Model fingerprint: 0x9e8aadb8
Variable types: 0 continuous, 167724 integer (167404 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+01]

User MIP start produced solution with objective 5079 (0.41s)
Loaded user MIP start with objective 5079

Presolve removed 2982 rows and 2119 columns (presolve time = 5s) ...
Presolve removed 2982 rows and 2119 columns (presolve time = 157s) ...
Presolve removed 5767 rows and 5341 columns (presolve time = 160s) ...
Presolve removed 5767 rows and 5341 columns (presolve time = 255s) ...
Presolve removed 5767 rows and 5341 columns (presolve time = 342s) ..

In [8]:
gurobi_prob.show_status()

Problem status: TIME_LIMIT
Obective value: 5042.0
